In [1]:
# 서울버스정류장
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()

cursor = mydb.cursor(prepared=True)

# 2021.09.25일 수집 완료 (더 이상 수집 필요없음)
query = ("select seoulsttsName from public_seoulbusstts ps where 10026 <= seq and seq <= 10921")

cursor.execute(query)

location_list = []

for i in cursor:
    location_list.append(i)
    
now = datetime.now()
nowDate = now.strftime('%Y%m%d')

cnt = 1
for i in location_list:
    seoulsttsName = i
    
    characters = "(',')"

    join_data = ''.join( x for x in seoulsttsName if x not in characters)
    print("조회날짜 : {} | 순번 : {} | start {} public_seoulbusstts".format(nowDate, cnt, join_data))
    url = 'http://ws.bus.go.kr/api/rest/stationinfo/getStationByName'
    queryParams = '?' + urlencode({
        quote_plus('serviceKey') : public_serviceKey(),
        quote_plus('stSrch') : join_data
    })

    response = requests.get(url + queryParams).text.encode('utf-8')
    xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

    rows = xmlobj.findAll('itemList')
    resultCode = xmlobj.findAll('headerCd')

    if resultCode[0].text == '7':
        print("{} 번째부터 조회수 제한이 걸렸습니다.".format(seoulsttsName))
        mydb.close()
        break
    cnt += 1
    for j in rows:
        item = j
        arsId = item.findAll('arsId')
        stId = item.findAll('stId')
        stNm = item.findAll('stNm')
        lat = item.findAll('tmY')
        lng = item.findAll('tmX')
        
        try:
            sql = "INSERT INTO public_seoulbusarsid_info (arsId, stId, stNm, lat, lng) VALUES (?,?,?,?,?)"
            val = (arsId[0].text, stId[0].text, stNm[0].text, lat[0].text, lng[0].text)
        except IndexError:
            print(item)
            sql = "INSERT INTO public_seoulbusarsid_info (arsId, stId, stNm, lat, lng) VALUES (?,?,?,?,?)"
            val = (arsId[0].text, stId[0].text, stNm[0].text, lat[0].text, lng[0].text)
            
        cursor.execute(sql, val)
        mydb.commit()

조회날짜 : 20210925 | 순번 : 1 | start 양재전화국 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 2 | start 대치2동주민센터.우래옥앞 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 3 | start 봉은사역.삼성래미안아파트 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 4 | start 강남구민체육관 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 5 | start LPG가스충전소 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 6 | start 삼성서울병원사거리 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 7 | start 삼성서울병원후문 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 8 | start 수서역 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 9 | start 일원1동주민센터 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 10 | start 일원지구대 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 11 | start 동영문화센터 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 12 | start 영희초등학교 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 13 | start 삼성서울병원정문건너편 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 14 | start 일원역 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 15 | start 노블치과.누네안과병원 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 16 | start 대왕파출소 public_seoulbusstts
조회날짜 : 2

조회날짜 : 20210925 | 순번 : 133 | start 잠실트리지움아파트앞 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 134 | start 잠실역.롯데월드 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 135 | start 갤러리아팰리스 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 136 | start 잠실트리지움 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 137 | start 영동일고레이크팰리스앞 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 138 | start 영동일고트리지움APT정문앞 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 139 | start 잠실트리지움 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 140 | start 잠실119안전센터 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 141 | start 잠실종합운동장 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 142 | start 아시아선수촌아파트 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 143 | start 아주중학교 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 144 | start 종합운동장사거리 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 145 | start 잠실새내역4번출구 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 146 | start 영빌딩앞 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 147 | start 잠실우성4차현대아파트 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 148 | start 잠

조회날짜 : 20210925 | 순번 : 263 | start 비호아파트 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 264 | start 인애가요양병원 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 265 | start 마천1동주민센터 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 266 | start 마천2동주민센터 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 267 | start 거여역3번출구.현대2차아파트 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 268 | start 마천1동주민센터 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 269 | start 송파상운종점 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 270 | start 비호아파트 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 271 | start 남한산성입구 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 272 | start 송파구립마천청소년센터 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 273 | start 송파구립마천청소년센터 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 274 | start 송파보훈회관.금호어울림2차 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 275 | start 송파보훈회관.금호어울림2차 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 276 | start 마천동금호어울림1차아파트 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 277 | start 마천동금호어울림1차아파트 public_seoulbusstts
조회날짜 : 20210925 

조회날짜 : 20210925 | 순번 : 390 | start 위례동주민센터.위례송파푸르지오 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 391 | start 힐스테이트송파.송파더센트레 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 392 | start 송파더센트레.힐스테이트송파 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 393 | start 복정역 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 394 | start 세계로병원 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 395 | start 세계로병원 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 396 | start 호반써밋송파1차 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 397 | start 호반써밋송파2차 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 398 | start 위례포레샤인13단지 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 399 | start 위례포레샤인13단지 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 400 | start 남한산성입구 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 401 | start 아산병원정문앞 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 402 | start 갑을.미성아파트 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 403 | start 송파배드민턴체육관 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 404 | start 위례31단지정문 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 405 | 

조회날짜 : 20210925 | 순번 : 517 | start 천호역 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 518 | start 천호역 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 519 | start 강동공영차고지 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 520 | start 강동공영차고지 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 521 | start 고덕리엔파크1단지 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 522 | start 고덕리엔파크1단지 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 523 | start 해뜨는주유소 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 524 | start 해뜨는주유소 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 525 | start 고덕천.상일동역교차로 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 526 | start 고덕천.상일동역교차로 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 527 | start 고덕주공3단지 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 528 | start 강명초등학교 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 529 | start 고덕주공6단지 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 530 | start 고덕리엔파크3단지 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 531 | start 상일동산.롯데캐슬베네루체 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 532 | start 상일동산.롯데캐슬베네

조회날짜 : 20210925 | 순번 : 645 | start 굽은다리사거리 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 646 | start 굽은다리사거리 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 647 | start 십자성마을 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 648 | start 십자성마을 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 649 | start 길동역1번출구 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 650 | start 길동역3번출구 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 651 | start 예닮요양병원 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 652 | start 예닮요양병원 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 653 | start 혜림교회앞 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 654 | start 동명약국앞 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 655 | start 둔촌동역4번출구 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 656 | start 둔촌동역2번출구 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 657 | start 둔촌종합상가 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 658 | start 둔촌종합상가 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 659 | start 고덕주공6단지상일여고 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 660 | start 고덕주공6단지상일여고 public_seoulbu

조회날짜 : 20210925 | 순번 : 773 | start 둔촌고앞 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 774 | start 한영중고앞사거리 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 775 | start 보훈병원 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 776 | start 고덕1동주민센터.두레근린공원 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 777 | start 고덕역1번출구 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 778 | start 명일신동아아파트 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 779 | start 강일리버파크3단지308동 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 780 | start 둔촌주공아파트1.4단지후문 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 781 | start 상일동역3.4번출구 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 782 | start 한산중학교 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 783 | start 주양쇼핑 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 784 | start 고덕리엔파크1단지 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 785 | start 둔촌역 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 786 | start 고덕역1번출구 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 787 | start 고덕1동주민센터.두레근린공원 public_seoulbusstts
조회날짜 : 20210925 | 순번 : 788 | start